In [19]:
import pandas as pd
import numpy as np

In [20]:
# Show all columns (instead of cascading columns in the middle)
pd.set_option("display.max_columns", None)
# Don't show numbers in scientific notation
pd.set_option("display.float_format", "{:.2f}".format)

In [21]:
# Read Pickle and Parquet
df_1 = pd.read_pickle('listings_project.pkl')
df_2 = pd.read_parquet('calendar_project.parquet',engine='pyarrow')

In [22]:
df_1.head()

,id,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,instant_bookable,reviews_per_month,price_in_euros,price_per_person,minimum_price,discount_per_5_days_booked,discount_per_10_days_booked,discount_per_30_and_more_days_booked,service_cost
0,23726706,0.95,f,1,1,IJburg - Zeeburgereiland,52.35,4.98,Private room,2,1.00,1.00,6,$88.00,2,14,t,0,6,20,66,78,11,3,4.99,t,1.53,None,$44,$176,5%,11%,16%,$4.99
1,35815036,1.00,t,1,1,Noord-Oost,52.42,4.96,Entire home/apt,2,NaN,1.00,5,$105.00,3,100,t,4,6,12,243,95,36,6,4.96,f,2.65,None,$52.5,$315,5%,12%,16%,$4.99
2,31553121,1.00,f,1,1,Noord-West,52.43,4.92,Entire home/apt,4,1.00,3.00,3,$152.00,2,60,t,0,3,3,3,82,26,1,4.74,f,2.02,None,$38,$304,7%,11%,22%,$4.99
3,34745823,0.94,f,3,3,Gaasperdam - Driemond,52.30,5.01,Entire home/apt,2,1.00,2.00,8,$87.00,2,1125,t,5,20,26,290,39,4,0,4.87,f,1.08,None,$43.5,$174,6%,10%,15%,$4.99
4,44586947,0.88,t,0,0,Gaasperdam - Driemond,52.31,5.03,Private room,4,2.00,3.00,4,$160.00,2,31,t,9,32,62,152,15,12,3,5.00,f,0.68,None,$40,$320,9%,22%,20%,$4.99


## Preprocessing the Dataset

In [23]:
# Print column names, types, and non-null values
df_1.info(verbose=True,show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6165 entries, 0 to 6172
Data columns (total 34 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    6165 non-null   int64  
 1   host_acceptance_rate                  5365 non-null   float64
 2   host_is_superhost                     6165 non-null   object 
 3   host_listings_count                   6165 non-null   int64  
 4   host_total_listings_count             6165 non-null   int64  
 5   neighbourhood_cleansed                6165 non-null   object 
 6   latitude                              6165 non-null   float64
 7   longitude                             6165 non-null   float64
 8   room_type                             6165 non-null   object 
 9   accommodates                          6165 non-null   int64  
 10  bedrooms                              5859 non-null   float64
 11  beds             

This info printout provides a good overview of which columns we need to investigate further. 

In [25]:
def wrangle(df_1):
    
    
    
    ''' Change the columns host_is_superhost, instant_bookable, 
        and has_availability into a boolean data type for better data processing:
        Replace f and t with False and True
        Set the column as type bool
        Overwrite the old columns with the new values'''
    
    df_1["host_is_superhost"] = df_1["host_is_superhost"].replace({'F':False,'t':True}).astype('bool')
    df_1['instant_bookable'] = df_1['instant_bookable'].replace({'f':False,'t':True}).astype('bool')
    df_1['has_availability'] = df_1['has_availability'].replace({'f':False,'t':True}).astype('bool')
        
    '''Perform this four-step process to change each of the three discount_per_... columns into their proper format:

        Remove non-numeric characters, like the percent symbol, so you can perform mathematical calculations on the column
        Change the column into a float data type in order to convert the data into a ratio
        Multiply the whole column by 0.01 so you end up with a probability ratio instead of a percentage
        Overwrite the old discount_per_... column with this new column'''
        
    df_1['discount_per_5_days_booked'] = df_1['discount_per_5_days_booked'].str.replace('%','', regex=True).astype(float) * 0.01
    df_1['discount_per_10_days_booked'] = df_1['discount_per_10_days_booked'].str.replace('%','',regex=True).astype(float) * 0.01
    df_1['discount_per_30_and_more_days_booked'] =  df_1['discount_per_30_and_more_days_booked'].str.replace('%','',regex=True).astype(float) * 0.01
        
    '''All of these four columns have some special characters that you will need to remove 
            before you can change the dtype from object to float.
            Remove dollar signs and commas
            Convert to float'''
        
    df_1['price'] =  df_1['price'].str.replace('$','',regex=True).str.replace(',','',regex=True).astype(float)
    df_1['price_per_person'] = df_1['price_per_person'].str.replace('$','',regex=True).str.replace(',','',regex=True).astype(float)
    df_1['minimum_price'] =  df_1['minimum_price'].str.replace('$','',regex=True).str.replace(',','',regex=True).astype(float)
    df_1['service_cost'] = df_1['service_cost'].str.replace('$','',regex=True).str.replace(',','',regex=True).astype(float)
        
    '''The following column names need to be changed:
        price into price_in_dollar
        neighbourhood_cleansed into neighbourhood'''
        
    df_1 = df_1.rename(columns={'price':'price_in_dollar','neighbourhood_cleansed':'neighbourhood'})
    
    '''Taking a closer look at the neighbourhood and room_type columns reveals that these columns are assigned an object dtype. 
    We want them to be a category dtype. Please set the correct data type below.'''
    
    df_1 = df_1.astype({'neighbourhood':'category','room_type':'category'})
    
    '''We need to delete some columns that are irrelevant to our current use case. Those irrelevant columns are:
    host_listings_count
    host_total_listings_count
    availability_60
    availability_90
    availability_365
    number_of_reviews
    number_of_reviews_ltm
    reviews_per_month'''
    
    df_1 = df_1.drop(
        columns=[
            "host_listings_count",
            "host_total_listings_count",
            "availability_60",
            "availability_90",
            "availability_365",
            "number_of_reviews",
            "number_of_reviews_ltm",
            "reviews_per_month",
        ])
    
    '''The approach should reveal that this column contains no unique values and is thus empty. Please drop this column.'''
    
    df_1 = df_1.drop(columns=["price_in_euros"])
    
    '''DataFrame info() revealed that some listings have no reviews and an unknown host acceptance rate. 
    Most Airbnb users exclude such listings from their search results. To mimic this filtering approach, 
    please filter out any rows that do not have a review_scores_rating and without a host_acceptance_rate'''
    
    df_1 = df_1.dropna(subset=["review_scores_rating", "host_acceptance_rate"])
    
    '''After setting the right data types, you are often left with making some hard decisions and assumptions
    about any partially incomplete data in your working dataset. In this case, some beds and bedrooms have no properly assigned
    values. You can check this by running df_list.info(verbose=True, show_counts=True), 
    which will show that beds and bedrooms have some missing values.

   Let's try and make some simple assumptions based on the room_type assigned to the listing. 
   First, inspect which room types are found in the dataset.
   There are four room types. Let's make the assumption that the columns bedrooms and beds are potentially influenced by room_type.

    Therefore, we can make the following rules:

    If you have a "Private room" or "Shared room" as room_type, then we believe the listing only has one bedroom.
    If the listing has "Hotel room" or "Entire home/apt" as room_type, then we can divide the number of guests the listing accomodates by 2 and round up.
    If any of these numbers are missing, then we can leave it empty.'''
    
    def fill_empty_bedrooms(accommodates: int, bedrooms: int, room_type: str) -> int:
        if (room_type == "Private room") or (room_type == "Shared room"):
            return 1
        elif (room_type == "Hotel room") or (room_type == "Entire home/apt"):
            return np.ceil(accommodates / 2)
        else:
            return bedrooms
        df_1["bedrooms"] = df_1[["accommodates", "bedrooms", "room_type"]].apply(
        lambda x: fill_empty_bedrooms(x["accommodates"], x["bedrooms"], x["room_type"]),
        axis=1,
    ) 
        
    '''Thanks to our logic and assumptions, most listings now have a proper amount of defined rooms. 
    However, there are still a few listings without any number of rooms defined. 
    Remove all rows/entries that have an empty bedrooms, beds.'''
    
    df_1 = df_1.dropna(subset=["bedrooms", "beds"])
    
    '''Now that we have removed all the empty values, finally we can assign the dtype int instead of float to these two columns.

    Please set the columns beds and bedrooms as int.'''
    
    df_1["beds"] = df_1["beds"].astype("int")
    df_1["bedrooms"] = df_1["bedrooms"].astype("int")
        
    return df_1


In [26]:
df = wrangle(df_1)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4566 entries, 0 to 6172
Data columns (total 25 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   id                                    4566 non-null   int64   
 1   host_acceptance_rate                  4566 non-null   float64 
 2   host_is_superhost                     4566 non-null   bool    
 3   neighbourhood                         4566 non-null   category
 4   latitude                              4566 non-null   float64 
 5   longitude                             4566 non-null   float64 
 6   room_type                             4566 non-null   category
 7   accommodates                          4566 non-null   int64   
 8   bedrooms                              4566 non-null   int32   
 9   beds                                  4566 non-null   int32   
 10  amenities                             4566 non-null   int64   
 11  pric

In [28]:
df.head()

,id,host_acceptance_rate,host_is_superhost,neighbourhood,latitude,longitude,room_type,accommodates,bedrooms,beds,amenities,price_in_dollar,minimum_nights,maximum_nights,has_availability,availability_30,number_of_reviews_l30d,review_scores_rating,instant_bookable,price_per_person,minimum_price,discount_per_5_days_booked,discount_per_10_days_booked,discount_per_30_and_more_days_booked,service_cost
0,23726706,0.95,True,IJburg - Zeeburgereiland,52.35,4.98,Private room,2,1,1,6,88.00,2,14,True,0,3,4.99,True,44.00,176.00,0.05,0.11,0.16,4.99
2,31553121,1.00,True,Noord-West,52.43,4.92,Entire home/apt,4,1,3,3,152.00,2,60,True,0,1,4.74,False,38.00,304.00,0.07,0.11,0.22,4.99
3,34745823,0.94,True,Gaasperdam - Driemond,52.30,5.01,Entire home/apt,2,1,2,8,87.00,2,1125,True,5,0,4.87,False,43.50,174.00,0.06,0.10,0.15,4.99
4,44586947,0.88,True,Gaasperdam - Driemond,52.31,5.03,Private room,4,2,3,4,160.00,2,31,True,9,3,5.00,False,40.00,320.00,0.09,0.22,0.20,4.99
5,15801253,0.86,True,Watergraafsmeer,52.35,4.96,Private room,3,1,3,1,90.00,2,1125,True,3,1,4.77,False,30.00,180.00,0.06,0.14,0.16,4.99


In [29]:
df_2.head()

,listing_id,date,available,price_in_dollar,minimum_nights,maximum_nights
0,23726706,2022-06-05,False,90.00,2,1125
1,23726706,2022-06-06,False,90.00,2,1125
2,23726706,2022-06-07,False,90.00,2,1125
3,23726706,2022-06-08,False,90.00,2,1125
4,23726706,2022-06-09,False,85.00,2,1125


You are looking to stay for a minimum of 3 days. Since you are unsure when it will be booked, you'd like to exclude all listing_ids that go below that threshold of 3 days no matter what time of year. This is because you think that bookings with a minimum stay of 3 days are more likely to have discount prices.

With these excluded, you would like to see the expected booking price for 5 days:

Using the helper code, create a conditional index for entries listing_id that are above the 3 day threshold
Remove them using .loc[conditional_index] over the conditional index.
Calculate the price of booking a listing for 5 days by multiplying the current day multiplied by 5, and assign this to a column called five_day_dollar_price

In [30]:
include_list = (
   df_2["minimum_nights"] >= 3
)

In [31]:
# Get all the listings with a minimum nights of 3+
# Use the include_list
df_2 = df_2.loc[include_list]

In [32]:
df_2["five_day_dollar_price"] = df_2["price_in_dollar"] * 5

C:\Users\HP\AppData\Local\Temp\ipykernel_2636\2676122290.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2["five_day_dollar_price"] = df_2["price_in_dollar"] * 5


Now let's transform our newly created DataFrame into a pivot table, where we aggregate our rows using the listing_id as the index, and the columns available and five_day_dollar_price as values.

In [33]:
calendar_summarizeddf = pd.pivot_table(
    data=df_2,
    index=["listing_id"],
    values=["available", "five_day_dollar_price"],
    aggfunc=np.mean,  # The default aggregation function used
    # for merging multiple related rows of data.
)

calendar_summarizeddf.head(3)

,available,five_day_dollar_price
listing_id,,
2818,0.21,346.90
44391,0.00,1200.00
49552,0.46,1162.50


### Let make a pivot table that states the maximum price_in_dollar for every Airbnb listing

In [34]:
temp_sum_df = pd.pivot_table(
    data=df_2,
    index=["listing_id"],
    values=["price_in_dollar"],
    aggfunc=np.max,  # The default aggregation function used
    # for merging multiple related rows of data.
)

temp_sum_df.head(3)

,price_in_dollar
listing_id,
2818,80.00
44391,240.00
49552,300.00


## Merging

In [35]:
final_df = pd.merge(
    df_1,
    calendar_summarizeddf,
    left_on=["id"],
    right_on=["listing_id"],
    how="inner",
)
final_df.head(3)

,id,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,neighbourhood_cleansed,latitude,longitude,room_type,accommodates,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,instant_bookable,reviews_per_month,price_in_euros,price_per_person,minimum_price,discount_per_5_days_booked,discount_per_10_days_booked,discount_per_30_and_more_days_booked,service_cost,available,five_day_dollar_price
0,35815036,1.00,True,1,1,Noord-Oost,52.42,4.96,Entire home/apt,2,NaN,1.00,5,105.00,3,100,True,4,6,12,243,95,36,6,4.96,False,2.65,None,52.50,315.00,0.05,0.12,0.16,4.99,0.66,528.89
1,19572024,1.00,True,2,2,Watergraafsmeer,52.31,4.91,Entire home/apt,6,3.00,6.00,14,279.00,3,300,True,6,10,30,298,126,23,3,4.69,False,2.13,None,46.50,837.00,0.09,0.16,0.14,4.99,0.82,1496.55
2,2973384,0.38,True,1,1,Watergraafsmeer,52.31,4.91,Entire home/apt,5,3.00,3.00,7,185.00,6,21,True,0,0,5,13,7,2,0,4.83,False,0.15,None,37.00,1110.00,0.06,0.12,0.18,4.99,0.04,941.30


Now, let's perform a groupby where we look at the median values of five_day_dollar_price and review_scores_rating with respect to the room_type. Do these results match your intuition?

In [39]:
final_df.groupby(by=["room_type"])[
    [
        "review_scores_rating",
        "five_day_dollar_price",
    ]
].median().sort_values(by='five_day_dollar_price',ascending=False)

,review_scores_rating,five_day_dollar_price
room_type,,
Hotel room,4.56,1139.81
Entire home/apt,4.88,950.82
Shared room,4.60,724.11
Private room,4.79,708.46


You might have expected that shared rooms are the cheapest and thus have the lowest rating with respect to median scores. The same can't be said for the most expensive option — a hotel room. Will this influence your future considerations when booking 🤔?

(But before you let this influence your decisions too much, it might be better to assume that this data might be biased in favor of Airbnb and not hotels in general. 🤷)